In [1]:
import json
import os

In [2]:
with open("./baselines/evaluation/evaluation_baselines.json", "r") as f:
    baseline_results = json.load(f)

with open("./outputs/evaluation/evaluation_logic_programs_gemini.json", "r") as f:
    logic_lm_results = json.load(f)


dataset_names = ("ProntoQA", "ProofWriter", "FOLIO", "LogicalDeduction", "AR-LSAT")


In [20]:
|dataset | info | Direct | CoT| Logic-LM (backup) |
| --- | --- | --- | ---  | --- | --- | --- |
|  ProntoQA | best acc |  79.96 | 99.34 | 97.40
|  ProntoQA | best model | gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0409   (CoT)|
|  ProofWriter | best acc |  56.67 | 74.11 | 79.73|
|  ProofWriter | best model |gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0409 | gemini-1.5-pro-preview-0409   (CoT)|
|  FOLIO | best acc |  66.67 | 77.61 | 82.67|
|  FOLIO | best model |gemini-1.5-flash-preview-0514 | gemini-1.5-pro-preview-0409 | gemini-1.5-pro-preview-0409   (CoT)|
|  LogicalDeduction | best acc |  59.00 | 69.00 | 84.67|
|  LogicalDeduction | best model |gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0409 | gemini-1.5-pro-preview-0514   (random)|
|  AR-LSAT | best acc |  28.14 | 25.54 | 38.53|
|  AR-LSAT | best model |gemini-1.5-pro-preview-0514 | gemini-1.0-pro-vision-001 | gemini-1.5-pro-preview-0514   (Direct)|

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (2530878615.py, line 4)

|dataset | info | Direct | CoT| Logic-LM (backup) |
| --- | --- | --- | ---  | --- | 
|  ProntoQA | best acc |  79.96 | 99.34 | 97.40
|  ProntoQA | best model | gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0409   (CoT)|
|  ProofWriter | best acc |  56.67 | 74.11 | 79.73|
|  ProofWriter | best model |gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0409 | gemini-1.5-pro-preview-0409   (CoT)|
|  FOLIO | best acc |  66.67 | 77.61 | 82.67|
|  FOLIO | best model |gemini-1.5-flash-preview-0514 | gemini-1.5-pro-preview-0409 | gemini-1.5-pro-preview-0409   (CoT)|
|  LogicalDeduction | best acc |  59.00 | 69.00 | 84.67|
|  LogicalDeduction | best model |gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0409 | gemini-1.5-pro-preview-0514   (random)|
|  AR-LSAT | best acc |  28.14 | 25.54 | 38.53|
|  AR-LSAT | best model |gemini-1.5-pro-preview-0514 | gemini-1.0-pro-vision-001 | gemini-1.5-pro-preview-0514   (Direct)|

In [12]:
logic_lm_results

{'gemini-1.0-pro-vision-001': {'ProntoQA': {'random': {'Overall_Accuracy': 0.774,
    'Executable_Rate': 1.0,
    'Executable_Accuracy': 0.774},
   'Direct': {'Overall_Accuracy': 0.774,
    'Executable_Rate': 1.0,
    'Executable_Accuracy': 0.774},
   'CoT': {'Overall_Accuracy': 0.774,
    'Executable_Rate': 1.0,
    'Executable_Accuracy': 0.774}},
  'ProofWriter': {'random': {'Overall_Accuracy': 0.6126878130217028,
    'Executable_Rate': 0.6444073455759599,
    'Executable_Accuracy': 0.7616580310880829},
   'Direct': {'Overall_Accuracy': 0.5542570951585977,
    'Executable_Rate': 0.6444073455759599,
    'Executable_Accuracy': 0.7616580310880829},
   'CoT': {'Overall_Accuracy': 0.6928213689482471,
    'Executable_Rate': 0.6444073455759599,
    'Executable_Accuracy': 0.7616580310880829}},
  'FOLIO': {'random': {'Overall_Accuracy': 0.53,
    'Executable_Rate': 0.485,
    'Executable_Accuracy': 0.6804123711340206},
   'Direct': {'Overall_Accuracy': 0.655,
    'Executable_Rate': 0.48,
    

In [13]:
def print_Logic_LM_table(logic_lm_results, dataset_names, backup='CoT'):
    for dataset in dataset_names:

        table_header = f"|{dataset}|"
        table_header += "|".join(logic_lm_results.keys()) 
        table_header += "|\n |---|---|---|---|---| \n"

        table_rows = []
        for metric in ['Overall_Accuracy', 'Executable_Rate', 'Executable_Accuracy']:
            row = f"|{metric}|"
            for model_name in logic_lm_results.keys():
                try:
                    row += f"{100 * logic_lm_results[model_name][dataset][backup][metric]:.2f}"
                    if logic_lm_results[model_name][dataset][backup]['Overall_Accuracy'] == 0:
                        print(model_name, dataset, backup)
                except:
                    row += "---"
                row += "|"
            table_rows.append(row)

        table_string = table_header + "\n".join(table_rows)
        print(table_string)
        print()

print_Logic_LM_table(logic_lm_results, dataset_names)

In [19]:
def print_best_vs_best_table(baseline_results, logic_lm_results, dataset_names):
    for dataset in dataset_names:
        max = -1
        base_direct = -1
        base_cot = -1
        
        max_model = ""
        base_direct_model = ""
        base_cot_model = ""

        for model_name in baseline_results.keys():
            for backup in ["random", "Direct", "CoT"]:
                acc = logic_lm_results[model_name][dataset][backup]['Overall_Accuracy']
                if acc > max:
                    max = acc
                    max_model = f"{model_name}   {backup}"
            direct_acc = baseline_results[model_name][dataset]['Direct']['Average_EM_score']
            cot_acc = baseline_results[model_name][dataset]['CoT']['Average_EM_score']
            if direct_acc > base_direct:
                base_direct = direct_acc
                base_direct_model = model_name
            if cot_acc > base_cot:
                base_cot = cot_acc
                base_cot_model = model_name

        row_acc = " | ".join([f"{100*base_direct:.2f}", f"{100*base_cot:.2f}", f"{100*max:.2f}"])
        row_model = " | ".join([base_direct_model, base_cot_model, max_model])

        print(row_acc)        
        print(row_model)
        
print_best_vs_best_table(baseline_results, logic_lm_results, dataset_names)

79.96 | 99.34 | 97.40
gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0409   CoT
56.67 | 74.11 | 79.73
gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0409 | gemini-1.5-pro-preview-0409   CoT
66.67 | 77.61 | 82.67
gemini-1.5-flash-preview-0514 | gemini-1.5-pro-preview-0409 | gemini-1.5-pro-preview-0409   CoT
59.00 | 69.00 | 84.67
gemini-1.5-pro-preview-0514 | gemini-1.5-pro-preview-0409 | gemini-1.5-pro-preview-0514   random
28.14 | 25.54 | 38.53
gemini-1.5-pro-preview-0514 | gemini-1.0-pro-vision-001 | gemini-1.5-pro-preview-0514   Direct


In [7]:
def print_all_gemini_results(baseline_results, logic_lm_results, dataset_names):
    for dataset in dataset_names:
        for model_name in baseline_results.keys():
            acc = logic_lm_results[model_name][dataset]['Overall_Accuracy']
            direct_acc = baseline_results[model_name][dataset]['Direct']['Average_EM_score']
            cot_acc = baseline_results[model_name][dataset]['CoT']['Average_EM_score']
            if acc > max:
                max = acc
                max_model = model_name
            if direct_acc > base_direct:
                base_direct = direct_acc
                base_direct_model = model_name
            if cot_acc > base_cot:
                base_cot = cot_acc
                base_cot_model = model_name

        row_acc = " | ".join([f"{100*base_direct:.2f}", f"{100*base_cot:.2f}", f"{100*max:.2f}"])
        row_model = " | ".join([base_direct_model, base_cot_model, max_model])

        print(row_acc)        
        print(row_model)